<a href="https://colab.research.google.com/github/Noorvh/Thesis_Sign-Language-Translator/blob/main/Data_preprocessing_TEST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Step 1: Load video and gloss data, and prepare for classification

1.1 load the segmented gloss videos

1.2 extract labels for data; get data statistics

1.3 turn videos into sets of images; extract frames for each video

1.4 turn images into numpy arrays

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
#1.1
!pip install ffmpeg

from google.colab.patches import cv2_imshow
import pandas as pd
import ffmpeg
import numpy as np
import cv2 as cv
import re
import math
from subprocess import check_call, PIPE, Popen
import shlex

  Preparing metadata (setup.py) ... done
  Created wheel for ffmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6080 sha256=0ed7ca5fe085a3e0bb6baf66e303e5e281e8e6ee04c7f0261182f59f479ee361
  Stored in directory: /root/.cache/pip/wheels/8e/7a/69/cd6aeb83b126a7f04cbe7c9d929028dc52a6e7d525ff56003a
Successfully built ffmpeg


In [8]:
gloss = pd.read_csv("drive/My Drive/Thesis/Data/Test/CNGT0095_r3.csv", header=None, names = ['signer', 'start time', 'start seconds', 'stop time', 'stop seconds', 'value', 'gloss'])
sentences = pd.read_csv("drive/My Drive/Thesis/Data/Test/Gloss2TextData.csv")
sentences = sentences.drop(['Unnamed: 0'], axis=1)

gloss = gloss.dropna(axis=0)

#sort by start time?
#gloss = gloss.sort_values('start time', axis=0)

In [9]:
gloss.tail()

signer  start time start seconds  stop time stop seconds  \
GlossR S2 S001  03:28.2      208.20       03:29.0     209.00      00:00.8   
          S001  03:29.1      209.08       03:29.2     209.16      00:00.1   
          S001  03:29.2      209.16       03:29.4     209.40      00:00.2   
          S001  03:29.8      209.80       03:30.5     210.52      00:00.7   
          S001  03:30.6      210.60       03:31.3     211.28      00:00.7   

                value  gloss  
GlossR S2 S001   0.80   AUTO  
          S001   0.08     PT  
          S001   0.24    MEE  
          S001   0.72  ERUIT  
          S001   0.68  WEG-C

In [35]:
## We get all the unique glosses, so we can sort the videos by gloss

unique_glosses = dict()
gloss_count = dict()
gloss_counter = dict()

for item in gloss['gloss']:
  if item not in unique_glosses:
    n = len(unique_glosses)
    unique_glosses[item] = n
    gloss_count[item] = 0
    gloss_counter[item] = 0
  gloss_count[item] = gloss_count[item]+1

#unique_glosses.sort()

print(unique_glosses)
print("Nr of unique glosses : " + str(len(unique_glosses)))

print(gloss_count)
print(gloss_counter)

{'BEDOELEN': 0, 'GEHANDICAPT-A': 1, 'ROLLATOR': 2, 'BE_HC_0': 3, 'WANDELEN': 4, 'PO': 5, 'BIJ-B': 6, 'AANPASSEN': 7, 'GROEP-A': 8, 'LUISTEREN': 9, 'MAAR': 10, 'DANSEN-B': 11, '%': 12, 'WEG-C': 13, 'BLIJVEN': 14, 'VOELEN-A?': 15, 'PT': 16, 'DISCUSSIEREN': 17, 'APPLAUS-A': 18, 'NEE-C': 19, 'DOOF-A': 20, 'NIET-B': 21, 'ANDERS-B': 22, 'ANDERE-KANT-A': 23, 'SLECHT-B': 24, 'HOREN-A': 25, 'WEL': 26, 'KLEIN-B': 27, 'ZIEN-A': 28, 'GEHOORAPPARAAT': 29, 'OORDOP': 30, 'ZELF': 31, 'NIKS-C': 32, 'ZOEKEN': 33, 'PV': 34, '1-A': 35, 'BETEKENEN': 36, 'HEBBEN-A': 37, 'EIGEN-A': 38, 'EILAND': 39, 'BEZOEK-A': 40, 'ONTMOETEN-A': 41, 'HALLO': 42, 'GEBAREN-A': 43, 'LOPEN-A': 44, 'NAAR': 45, 'KIJKEN-A': 46, 'PROBLEEM': 47, 'MOEILIJK': 48, 'AANSLUITEN': 49, 'WEG-B?': 50, 'VRIJ-C': 51, 'MENS': 52, 'UITWISSELEN-F': 53, 'ALS-B': 54, 'NAAST': 55, 'MOETEN-A': 56, 'BINNEN': 57, 'VEEL-B': 58, 'DOEL': 59, 'DRAW_5': 60, 'PAKKEN-B': 61, 'IN-A': 62, 'PAKKEN-A': 63, 'PRATEN-A': 64, 'INPAKKEN': 65, 'VROEGER-A': 66, 'POND': 

In [38]:
# for each unique gloss we make a folder in our videos directory, and in our images directory

import os, shutil

framedir_name = "drive/My Drive/Thesis/Data/Test/GlossFrames"
viddir_name = "drive/My Drive/Thesis/Data/Test/GlossVideos"

for i in unique_glosses:
  os.mkdir(os.path.join(framedir_name , i))
  os.mkdir(os.path.join(viddir_name , i))

In [39]:
re_metadata = re.compile('Duration: (\d{2}):(\d{2}):(\d{2})\.\d+,.*\n.* (\d+(\.\d+)?) fps')

def get_metadata(filename):
    '''
    Get video metadata using ffmpeg
    '''
    p1 = Popen(["ffmpeg", "-hide_banner", "-i", filename], stderr=PIPE, universal_newlines=True)
    output = p1.communicate()[1]
    matches = re_metadata.search(output)
    if matches:
        video_length = int(matches.group(1)) * 3600 + int(matches.group(2)) * 60 + int(matches.group(3))
        video_fps = float(matches.group(4))
        # print('video_length = {}\nvideo_fps = {}'.format(video_length, video_fps))
    else:
        raise Exception("Can't parse required metadata")
    return video_length, video_fps

def split_cut(path, outputpath, filename, ext, starts, stops, glosses):
    '''
    Split video by cutting and re-encoding: accurate but very slow
    Adding "-c copy" speed up the process but causes imprecise chunk durations
    Reference: https://stackoverflow.com/a/28884437/1862500
    '''
    split_count = len(starts)

    input = '{}{}.{}'.format(path, filename, ext)

    # parse meta data
    video_length, video_fps = get_metadata(input)

    # split the video by gloss
    output = []
    for i in range(split_count):
        split_start = starts[i]
        split_size = stops[i] - starts[i]
        counter = gloss_counter[glosses[i]]
        output_path = '{}{}/{}.{}'.format(outputpath, glosses[i], counter, ext)
        gloss_counter[glosses[i]] = gloss_counter[glosses[i]]+1
        cmd = 'ffmpeg -hide_banner -loglevel panic -ss {} -t {} -i "{}" -y "{}"'.format(
            split_start,
            split_size,
            input,
            output_path
        )
        # print(cmd)
        check_call(shlex.split(cmd), universal_newlines=True)
        output.append(output_path)
    return output

In [40]:
## Here I split the data video into chunks of each gloss.

## Need to add: way to split video into left or right based on who's speaking with ffmpeg crop filter

path = "drive/My Drive/Thesis/Data/Test/"
glossvideopath = "drive/My Drive/Thesis/Data/Test/GlossVideos/"
filename = "CNGT0095video"
ext = 'mpg'

split_cut(path, glossvideopath, filename, ext, gloss['start time'], gloss['stop time'], gloss['gloss'])

['drive/My Drive/Thesis/Data/Test/GlossVideos/BEDOELEN/0.mpg',
 'drive/My Drive/Thesis/Data/Test/GlossVideos/GEHANDICAPT-A/0.mpg',
 'drive/My Drive/Thesis/Data/Test/GlossVideos/ROLLATOR/0.mpg',
 'drive/My Drive/Thesis/Data/Test/GlossVideos/BE_HC_0/0.mpg',
 'drive/My Drive/Thesis/Data/Test/GlossVideos/WANDELEN/0.mpg',
 'drive/My Drive/Thesis/Data/Test/GlossVideos/PO/0.mpg',
 'drive/My Drive/Thesis/Data/Test/GlossVideos/BIJ-B/0.mpg',
 'drive/My Drive/Thesis/Data/Test/GlossVideos/AANPASSEN/0.mpg',
 'drive/My Drive/Thesis/Data/Test/GlossVideos/GROEP-A/0.mpg',
 'drive/My Drive/Thesis/Data/Test/GlossVideos/GEHANDICAPT-A/1.mpg',
 'drive/My Drive/Thesis/Data/Test/GlossVideos/PO/1.mpg',
 'drive/My Drive/Thesis/Data/Test/GlossVideos/LUISTEREN/0.mpg',
 'drive/My Drive/Thesis/Data/Test/GlossVideos/MAAR/0.mpg',
 'drive/My Drive/Thesis/Data/Test/GlossVideos/DANSEN-B/0.mpg',
 'drive/My Drive/Thesis/Data/Test/GlossVideos/%/0.mpg',
 'drive/My Drive/Thesis/Data/Test/GlossVideos/WEG-C/0.mpg',
 'drive/My 

In [43]:
def get_frames(filename, n_frames= 1):
    frames = []
    v_cap = cv.VideoCapture(filename)
    v_len = int(v_cap.get(cv.CAP_PROP_FRAME_COUNT))
    frame_list= np.linspace(0, v_len-1, n_frames+1, dtype=np.int16)

    for fn in range(v_len):
        success, frame = v_cap.read()
        if success is False:
            continue
        if (fn in frame_list):
            frame = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
            frames.append(frame)
    v_cap.release()
    return frames, v_len

def store_frames(frames, path2store):
    for ii, frame in enumerate(frames):
        frame = cv.cvtColor(frame, cv.COLOR_RGB2BGR)
        path2img = os.path.join(path2store, "frame"+str(ii)+".jpg")
        cv.imwrite(path2img, frame)

In [46]:
for item in unique_glosses:
  sub_folder = item
  path2gloss = os.path.join(glossvideopath, sub_folder)

  listOfCategories = os.listdir(path2gloss)
  listOfCategories, len(listOfCategories)

  extension = ".mpg"
  n_frames = 8
  for root, dirs, files in os.walk(path2gloss, topdown=False):
      for name in files:
          if extension not in name:
              continue
          path2vid = os.path.join(root, name)
          frames, vlen = get_frames(path2vid, n_frames= n_frames)
          path2store = path2vid.replace(extension, "")
          print(path2store)
          os.makedirs(path2store, exist_ok= True)
          store_frames(frames, path2store)
      print("-"*50)

drive/My Drive/Thesis/Data/Test/GlossVideos/BEDOELEN/0
--------------------------------------------------
drive/My Drive/Thesis/Data/Test/GlossVideos/GEHANDICAPT-A/0
drive/My Drive/Thesis/Data/Test/GlossVideos/GEHANDICAPT-A/1
drive/My Drive/Thesis/Data/Test/GlossVideos/GEHANDICAPT-A/2
drive/My Drive/Thesis/Data/Test/GlossVideos/GEHANDICAPT-A/3
drive/My Drive/Thesis/Data/Test/GlossVideos/GEHANDICAPT-A/4
drive/My Drive/Thesis/Data/Test/GlossVideos/GEHANDICAPT-A/5
drive/My Drive/Thesis/Data/Test/GlossVideos/GEHANDICAPT-A/6
drive/My Drive/Thesis/Data/Test/GlossVideos/GEHANDICAPT-A/7
drive/My Drive/Thesis/Data/Test/GlossVideos/GEHANDICAPT-A/8
drive/My Drive/Thesis/Data/Test/GlossVideos/GEHANDICAPT-A/9
drive/My Drive/Thesis/Data/Test/GlossVideos/GEHANDICAPT-A/10
drive/My Drive/Thesis/Data/Test/GlossVideos/GEHANDICAPT-A/11
drive/My Drive/Thesis/Data/Test/GlossVideos/GEHANDICAPT-A/12
--------------------------------------------------
drive/My Drive/Thesis/Data/Test/GlossVideos/ROLLATOR/0
drive

## Now we have to turn the images into numpy arrays so it can be used in a pytorch classifier

In [52]:
!pip install Pillow

# Import the necessary libraries
from PIL import Image
from numpy import asarray


size = 176, 144

# load the image and convert into
# numpy array
extension = ".jpg"

for root, dirs, files in os.walk(glossvideopath, topdown=False):
      for name in files:
          if extension not in name:
              continue
          path2frame = os.path.join(root, name)
          img = Image.open(path2frame)
          img_resized = img.resize(size, Image.ANTIALIAS)

          # asarray() class is used to convert
          # PIL images into NumPy arrays
          numpydata = asarray(img_resized)

          #print(type(numpydata))          # <class 'numpy.ndarray'>
          #print(numpydata.shape)          #  shape

<ipython-input-52-b5d0f576c1d7>:18: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  img_resized = img.resize(size, Image.ANTIALIAS)


<class 'numpy.ndarray'>
(144, 176, 3)
<class 'numpy.ndarray'>
(144, 176, 3)
<class 'numpy.ndarray'>
(144, 176, 3)
<class 'numpy.ndarray'>
(144, 176, 3)
<class 'numpy.ndarray'>
(144, 176, 3)
<class 'numpy.ndarray'>
(144, 176, 3)
<class 'numpy.ndarray'>
(144, 176, 3)
<class 'numpy.ndarray'>
(144, 176, 3)
<class 'numpy.ndarray'>
(144, 176, 3)
<class 'numpy.ndarray'>
(144, 176, 3)
<class 'numpy.ndarray'>
(144, 176, 3)
<class 'numpy.ndarray'>
(144, 176, 3)
<class 'numpy.ndarray'>
(144, 176, 3)
<class 'numpy.ndarray'>
(144, 176, 3)
<class 'numpy.ndarray'>
(144, 176, 3)
<class 'numpy.ndarray'>
(144, 176, 3)
<class 'numpy.ndarray'>
(144, 176, 3)
<class 'numpy.ndarray'>
(144, 176, 3)
<class 'numpy.ndarray'>
(144, 176, 3)
<class 'numpy.ndarray'>
(144, 176, 3)
<class 'numpy.ndarray'>
(144, 176, 3)
<class 'numpy.ndarray'>
(144, 176, 3)
<class 'numpy.ndarray'>
(144, 176, 3)
<class 'numpy.ndarray'>
(144, 176, 3)
<class 'numpy.ndarray'>
(144, 176, 3)
<class 'numpy.ndarray'>
(144, 176, 3)
<class 'nump

KeyboardInterrupt: 

In [ ]:
# create csv in format:
#   {
#      'video': <video_tensor>,     # Shape: (C, T, H, W)
#      'audio': <audio_tensor>,     # Shape: (S)
#      'label': <action_label>,     # Integer defining class annotation
#      'video_name': <video_path>,  # Video file path stem
#      'video_index': <video_id>,   # index of video used by sampler
#      'clip_index': <clip_id>      # index of clip sampled within video
#   }